In [1]:
import casadi as ca
import numpy as np
from casadi.tools.graph import dotgraph
from IPython.display import Image

def draw_graph(expr):
    return Image(dotgraph(expr).create_png())

In [2]:
def rhs(x, u):
    s = 2170
    cbar = 17.5
    mass = 5.0e3
    iyy = 4.1e6
    tstat = 6.0e4
    dtdv = -38.0
    ze = 2.0
    cdcls = 0.042
    cla = 0.085
    cma = -0.022
    cmde = -0.016
    cmq = -16.0
    cmadot = -6.0
    cladot = 0.0
    rtod = 57.29578
    gd = 32.17
    
    thtl = u[0]
    elev_deg = u[1]
    xcg = u[2]
    land = u[3]
    
    vt = x[0]  # velocity, ft/s
    alpha = x[1]
    alpha_deg = rtod*alpha  # angle of attack, deg
    theta = x[2]  # pitch angle, rad
    q = x[3]  # pitch rate, rad/s
    h = x[4]  # altitude, ft
    pos = x[5]  # horizontal position from origin, ft (not used in dynamics)
    
    r0 = 2.377e-3
    tfac = 1.0 - 0.703e-5*h
    temperature = ca.if_else(h > 35000, 390.0, 519.0*tfac)
    rho = r0*(tfac**4.14)
    mach = vt/ca.sqrt(1.4*1716.3*temperature)
    qbar = 0.5*rho*vt**2
    
    qs = qbar*s
    salp = ca.sin(alpha)
    calp = ca.cos(alpha)
    gam = theta - alpha
    sgam = ca.sin(gam)
    cgam = ca.cos(gam)
    
    aero_p = ca.if_else(
        land,
        (1.0, 0.08, -0.20, 0.02, -0.05),
        (0.2, 0.016, 0.05, 0.0, 0.0))
    cl0 = aero_p[0]
    cd0 = aero_p[1]
    cm0 = aero_p[2]
    dcdg = aero_p[3]
    dcmg = aero_p[4]
    
    thr = (tstat + vt*dtdv)*ca.fmax(thtl, 0)
    cl = cl0 + cla*alpha_deg
    cm = dcmg + cm0 + cma*alpha_deg + cmde*elev_deg + cl*(xcg - 0.25)
    cd = dcdg + cd0 + cdcls*cl**2
    
    x_dot = ca.SX.zeros(6)
    x_dot[0] = (thr*calp - qs*cd)/mass - gd*sgam
    x_dot[1] = (-thr*salp - qs*cl + mass*(vt*q + gd*cgam))/(mass*vt + qs*cladot)
    x_dot[2] = q
    d = 0.5*cbar*(cmq*q + cmadot*x_dot[1])/vt
    x_dot[3] = (qs*cbar*(cm + d) + thr*ze)/iyy
    x_dot[4] = vt*sgam
    x_dot[5] = vt*cgam
    return x_dot

In [3]:
def constrain(s, vt, h, gamma):
    
    # s is our design vector:
    # s = [thtl, elev_deg, alpha]
    thtl = s[0]
    elev_deg = s[1]
    alpha = s[2]
    
    pos = 0  # we don't care what horiz. position we are at
    q = 0 # we don't want to be rotating, so no pitch-rate
    xcg = 0.25  # we assume xcg at 1/4 chord
    land = 0  # we assume we do not have flaps/gear deployed
    theta = alpha + gamma
    
    # vt, alpha, theta, q, h, pos
    x = ca.vertcat(vt, alpha, theta, q, h, pos)
    
    # thtl, elev_deg, xcg, land
    u = ca.vertcat(thtl, elev_deg, xcg, land)
    return x, u

def trim_cost(x, u):
    x_dot = rhs(x, u)
    return x_dot[0]**2 + 100*x_dot[1]**2 + 10*x_dot[3]**2

def objective(s, vt, h, gamma):
    x, u = constrain(s, vt, h, gamma)
    return trim_cost(x, u)

In [4]:
def solve_problem():
    s = ca.SX.sym('s', 3)
    nlp = {'x': s, 'f': objective(s, vt=500, h=0, gamma=0)}
    S = ca.nlpsol('S', 'ipopt', nlp)

    # s = [thtl, elev_deg, alpha]
    s0 = [0.293, 2.46, np.deg2rad(0.58)]
    res = S(x0=s0)
    return res['x']

In [5]:
solve_problem()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

DM([0.292673, 2.4607, 0.0101196])